In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# base_dir='/content/drive/MyDrive/ucd/'
base_dir='../../../folders/'

In [2]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
from sklearn.metrics import classification_report,accuracy_score
from sklearn.model_selection import train_test_split
from keras.models import Sequential,load_model
from keras.layers import Conv1D, Conv2D,MaxPooling1D,MaxPooling2D, Dropout,Dense,Flatten,Activation, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import scipy.stats as stats
from keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")



2024-08-21 14:09:35.884639: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-21 14:09:35.903253: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-21 14:09:35.903268: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-21 14:09:35.903749: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-21 14:09:35.906986: I tensorflow/core/platform/cpu_feature_guar

In [3]:
from tensorflow.keras import backend as K

def f1(y_true, y_pred):
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))

        recall = TP / (Positives+K.epsilon())
        return recall

    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

        precision = TP / (Pred_Positives+K.epsilon())
        return precision
    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [4]:
model=load_model(base_dir+"models/cnn.h5",custom_objects={'f1':f1})
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_16 (Ba  (None, 3840, 1)           4         
 tchNormalization)                                               
                                                                 
 conv1d_24 (Conv1D)          (None, 1916, 5)           55        
                                                                 
 activation_24 (Activation)  (None, 1916, 5)           0         
                                                                 
 max_pooling1d_24 (MaxPooli  (None, 958, 5)            0         
 ng1D)                                                           
                                                                 
 conv1d_25 (Conv1D)          (None, 949, 45)           2295      
                                                                 
 max_pooling1d_25 (MaxPooli  (None, 474, 45)          

2024-08-21 14:09:36.867380: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-21 14:09:36.883293: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-21 14:09:36.883393: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [5]:
for layer in model.layers:
    if layer.__class__.__name__ in ['Conv1D', 'Dense']:
        w = layer.get_weights()[0]
        layersize = np.prod(w.shape)
        print("{}: {}".format(layer.name, layersize))  # 0 = weights, 1 = biases
        if layersize > 4096:  # assuming that shape[0] is batch, i.e., 'None'
            print("Layer {} is too large ({}), are you sure you want to train?".format(layer.name, layersize))

conv1d_24: 50
conv1d_25: 2250
conv1d_26: 33750
Layer conv1d_26 is too large (33750), are you sure you want to train?
dense_11: 11100
Layer dense_11 is too large (11100), are you sure you want to train?


In [6]:
import hls4ml
import os
os.environ['PATH'] = '/tools/Xilinx/Vitis_HLS/2023.2/bin:' + os.environ['PATH']

In [7]:
hls_config = hls4ml.utils.config_from_keras_model(model, granularity='name')
hls_config['Model']['ReuseFactor'] = 1
hls_config['Model']['PipelineStyle'] = 'dataflow'
# hls_config['Model']['Precision'] = 'ap_fixed<8,1>'


for Layer in hls_config['LayerName'].keys():
    hls_config['LayerName'][Layer]['Strategy'] = 'Resource'
    hls_config['LayerName'][Layer]['ReuseFactor'] = 5
    
hls_config['LayerName']['conv1d_24']['Strategy'] = 'Resource'
hls_config['LayerName']['conv1d_24']['ReuseFactor'] = 1

hls_config['LayerName']['conv1d_25']['Strategy'] = 'Resource'
hls_config['LayerName']['conv1d_25']['ReuseFactor'] = 150

hls_config['LayerName']['conv1d_26']['Strategy'] = 'Resource'
hls_config['LayerName']['conv1d_26']['ReuseFactor'] = 450

hls_config['LayerName']['dense_11']['Strategy'] = 'Resource'
hls_config['LayerName']['dense_11']['ReuseFactor'] = 111

print(hls_config)

Interpreting Sequential
Topology:
Layer name: batch_normalization_16_input, layer type: InputLayer, input shapes: [[None, 3840, 1]], output shape: [None, 3840, 1]
Layer name: batch_normalization_16, layer type: BatchNormalization, input shapes: [[None, 3840, 1]], output shape: [None, 3840, 1]
Layer name: conv1d_24, layer type: Conv1D, input shapes: [[None, 3840, 1]], output shape: [None, 1916, 5]
Layer name: activation_24, layer type: Activation, input shapes: [[None, 1916, 5]], output shape: [None, 1916, 5]
Layer name: max_pooling1d_24, layer type: MaxPooling1D, input shapes: [[None, 1916, 5]], output shape: [None, 958, 5]
Layer name: conv1d_25, layer type: Conv1D, input shapes: [[None, 958, 5]], output shape: [None, 949, 45]
Layer name: max_pooling1d_25, layer type: MaxPooling1D, input shapes: [[None, 949, 45]], output shape: [None, 474, 45]
Layer name: activation_25, layer type: Activation, input shapes: [[None, 474, 45]], output shape: [None, 474, 45]
Layer name: conv1d_26, layer t

In [8]:
hls_model = hls4ml.converters.convert_from_keras_model(
    model,
    hls_config=hls_config,
    output_dir='cnn/',
    project_name='cnn',
    io_type='io_stream',
    backend='Vitis',
    part='xc7a200tfbg484-3' )

hls_model.compile()

Interpreting Sequential
Topology:
Layer name: batch_normalization_16_input, layer type: InputLayer, input shapes: [[None, 3840, 1]], output shape: [None, 3840, 1]
Layer name: batch_normalization_16, layer type: BatchNormalization, input shapes: [[None, 3840, 1]], output shape: [None, 3840, 1]
Layer name: conv1d_24, layer type: Conv1D, input shapes: [[None, 3840, 1]], output shape: [None, 1916, 5]
Layer name: activation_24, layer type: Activation, input shapes: [[None, 1916, 5]], output shape: [None, 1916, 5]
Layer name: max_pooling1d_24, layer type: MaxPooling1D, input shapes: [[None, 1916, 5]], output shape: [None, 958, 5]
Layer name: conv1d_25, layer type: Conv1D, input shapes: [[None, 958, 5]], output shape: [None, 949, 45]
Layer name: max_pooling1d_25, layer type: MaxPooling1D, input shapes: [[None, 949, 45]], output shape: [None, 474, 45]
Layer name: activation_25, layer type: Activation, input shapes: [[None, 474, 45]], output shape: [None, 474, 45]
Layer name: conv1d_26, layer t

In [9]:
hls_model.build(csim=False, synth=True, vsynth=True)


****** Vitis HLS - High-Level Synthesis from C, C++ and OpenCL v2023.2 (64-bit)
  **** SW Build 4023990 on Oct 11 2023
  **** IP Build 4028589 on Sat Oct 14 00:45:43 MDT 2023
  **** SharedData Build 4025554 on Tue Oct 10 17:18:54 MDT 2023
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.
    ** Copyright 2022-2023 Advanced Micro Devices, Inc. All Rights Reserved.

source /tools/Xilinx/Vitis_HLS/2023.2/scripts/vitis_hls/hls.tcl -notrace
INFO: [HLS 200-10] Running '/tools/Xilinx/Vitis_HLS/2023.2/bin/unwrapped/lnx64.o/vitis_hls'
INFO: [HLS 200-10] For user 'zahra' on host 'zahra' (Linux_x86_64 version 6.5.0-44-generic) on Wed Aug 21 14:09:41 +0330 2024
INFO: [HLS 200-10] On os Ubuntu 22.04.4 LTS
INFO: [HLS 200-10] In directory '/home/zahra/Desktop/SleepStage/hw/hls4ml/cnn_undersampling/cnn'
INFO: [HLS 200-2053] The vitis_hls executable is being deprecated. Consider using vitis-run --mode hls --tcl
Sourcing Tcl script 'build_prj.tcl'
INFO: [HLS 200-1510] Running: open_project c

{'CSynthesisReport': {'TargetClockPeriod': '5.00',
  'EstimatedClockPeriod': '6.411',
  'BestLatency': '233700',
  'WorstLatency': '234178',
  'IntervalMin': '61442',
  'IntervalMax': '234158',
  'BRAM_18K': '328',
  'DSP': '259',
  'FF': '179899',
  'LUT': '118720',
  'URAM': '0',
  'AvailableBRAM_18K': '730',
  'AvailableDSP': '740',
  'AvailableFF': '269200',
  'AvailableLUT': '134600',
  'AvailableURAM': '0'},
 'VivadoSynthReport': {'LUT': '51424',
  'FF': '151573',
  'BRAM_18K': '104',
  'DSP48E': '259'}}